## Segmentación de organos humanos sobre imagenes de resonancia magnetica mediante redes neuronales profundas.

<img src="https://i.ibb.co/vJTPZgN/banner.jpg">

### Imports:

In [0]:
#imports:
import glob
import cv2 as cv
import os
import sys
import matplotlib.pyplot as plt
from __future__ import print_function
import nibabel as nib
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
from skimage.segmentation import mark_boundaries
#pasar de keras a tensor flowvv
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from skimage.exposure import rescale_intensity
from keras.callbacks import History
from skimage import io
from data import load_train_data, load_test_data
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from tqdm import tqdm_notebook as tqdm

In [0]:
!pip install tensorflow

In [0]:
#path ejemplo /content/drive/My Drive/Colab Notebooks/Python/Proyectos/2019-2-vision-por-computadora-proyecto
#path ejemplo /content/drive/My Drive/Colab Notebooks/2019-2-vision-por-computadora-proyecto
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/Python/Proyectos/2019-2-vision-por-computadora-proyecto
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/Python/Proyectos/2019-2-vision-por-computadora-proyecto
/content/drive/My Drive/Colab Notebooks/Python/Proyectos/2019-2-vision-por-computadora-proyecto


### [Ingrese nombre de proyecto]

**Integrantes:**

- Jorge Andres Mogotocoro Fajardo - 2160075
- Mario Hernan Vallejo Huertas    - 2150403
- Bryan Daniel Alvarez            - 

### Objetivo general

- Segmentar el hígado a partir de escaneos de pacientes mediante visión por computadora.

### Objetivos especificos

-  [ - - - ]
-  [ - - - ]
-  [ - - - ]

### Preguntas por responder

- **Problema a resolver:**  
R: Segmentar el hígado a partir de escaneos de pacientes
- **¿Como se va resolver?**  
R: Basandonos en el metodo utilizado en el trabajo de investigacion ["U-Net: Convolutional Networks for Biomedical
Image Segmentation"](https://arxiv.org/pdf/1505.04597.pdf)
- **¿Por qué se necesita resolver este problema?**  
R: El desarrollo tecnológico ha propiciado un cambio asombroso en la medicina, proveyendo de procesos que explican el porqué de muchas enfermedades, de eventos que ocurren en el organismo humano por tanto nuestro proyecto pretende como objetivo a futuro dar a disposicion de la medicina una herramienta que en caso particular permita detectar anomalias en el higado basados en su forma no obstante las bases empleadas esperamos puedan ser de mucha utilidad no solo en detectar un higado si no por el contrario llegar a generalizarse.
- **¿Para que?**  
R: Nuestro proyecto en principio tiene el objetivo de convertirse en una herramienta lo suficientemente precisa para dictaminar desiciones criticas que puedan salvar y/o prevenir la vida de un paciente.
- **¿Para quien(es) seria util esta solucion?**  
R: 

### Documentacion del problema:



- [La importancia de la tecnologia en la medicina](https://prezi.com/wdfcq6jniuaf/la-importancia-de-la-tecnologia-en-la-medicina/)

[Pequeño resumen]

- [Nibabel library](https://nipy.org/nibabel/)

[¿Que es?]

- [U-Net: Convolutional Networks for Biomedical
Image Segmentation](https://prezi.com/wdfcq6jniuaf/la-importancia-de-la-tecnologia-en-la-medicina/)

[Pequeño resumen]

### Implementacion

#### Dataset

[Fuente](https://www.dropbox.com/s/8h2avwtk8cfzl49/ircad-dataset.zip?dl=0)

- El dataset cuenta con formato **nii.gz** (nibabel, [NIfTI](http://niftilib.sourceforge.net/pynifti/intro.html)) el cual hace referencia a datos de neuro imagenes (imagenes de resonancia magnetica estructural MRI). Es un nuevo formato de tados peropuesto por el  [**Grupo de Trabajo de Formato de Datos de NIfTI**](http://nifti.nimh.nih.gov/dfwg/beyond-nifti-1) como una "medida a corto plazo para facilitar la interacción de los paquetes de software de análisis de datos de IRM funcionales.
- Se dispone de 20 conjuntos de imagenes por resonancia magnéticas tomadas a 20 diferentes pacientes.
- Los 20 conjuntos de datos se dividen en 20 imagenes por resonancia magnética identificadas por **orig** y 20 conjunto de imagenes mostrando unicamente la zona donde se ubica el higado en su correspondiente imagen por resonancia magnética original, se identifican por **liver**.
- Cada conjunto de imagenes de resonancia magnetica tomadas para cada paciente tiene dimenciones de **512 de ancho** por **512 de alto** por la cantidad de tomas o **secciones** que fueron capturadas.

##### Exploracion de los datos

In [0]:
def plot_1(imagen):

  plt.axis('off')
  img = plt.imshow(imagen, cmap="gray")
  return img

In [0]:
data_path = 'raw/test/'
example_filename = os.path.join(data_path, 'ircad_e20_orig.nii.gz')
img_orig = nib.load(example_filename)
img_orig.shape

(512, 512, 225)

In [0]:
data_orig = img_orig.get_fdata()

In [0]:
interact(lambda index:plot_1(data_orig[:,:,index]),
         index = widgets.IntSlider(min=0,max=112,
                                   step=1))

interactive(children=(IntSlider(value=0, description='index', max=112), Output()), _dom_classes=('widget-inter…

<function __main__.<lambda>>

In [0]:
data_path = 'raw/test/'
example_filename = os.path.join(data_path, 'ircad_e14_liver.nii.gz')
img_liver = nib.load(example_filename)
img_liver.shape

(512, 512, 113)

In [0]:
data_liver = img.get_fdata()

In [0]:
interact(lambda index:plot_1(data_liver[:,:,index]),
         index = widgets.IntSlider(min=0,max=112,
                                   step=1))

interactive(children=(IntSlider(value=0, description='index', max=112), Output()), _dom_classes=('widget-inter…

<function __main__.<lambda>>

#### Video ilustrativo

In [0]:
def image_to_frames(image, i):
    if i < 10:
        f_name = "frames/frame000"+str(i)+".png"
    else:
        if i < 100:
            f_name = "frames/frame00"+str(i)+".png"
        else:
            f_name = "frames/frame0"+str(i)+".png"    
    cv.imwrite(f_name, image)

def frame_to_video(nombre_video):
    img_array = []
    for filename in glob.glob('frames/frame*.png'):
        img = cv.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)
 
    out = cv.VideoWriter(nombre_video,cv.VideoWriter_fourcc(*'DIVX'), 15, size)
    
    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()

In [0]:
!mkdir frames
for i in range(data_orig.shape[2]):
  image = data_orig[:,:,i]
  image_to_frames(image, i)

frame_to_video("data_orig.avi")

In [0]:
def image_to_frames_2(image, i):
    if i < 10:
        f_name = "frames/frame000"+str(i)+".png"
    else:
        if i < 100:
            f_name = "frames/frame00"+str(i)+".png"
        else:
            f_name = "frames/frame0"+str(i)+".png"

    plt.axis('off')
    plt.imshow(image, 'gray')    
    plt.savefig(f_name, dpi=300, bbox_inches='tight')

!mkdir frames
for i in tqdm( range(data_liver.shape[2]) ):
  image = data_liver[:,:,i]
  image_to_frames_2(image, i)

frame_to_video("data_liver.avi")

<img src="https://s5.gifyu.com/images/data_orig_2.gif">

#### Particion de los datos

In [0]:
def load_image(path, image):
  example_filename = os.path.join(path, image)
  img_orig = nib.load(example_filename)
  data_orig = img_orig.get_fdata()
  return data_orig

In [0]:
def plot_2(imagenes):
  interact(lambda index:plot_1(imagenes[:,:,index]),
         index = widgets.IntSlider(min=0,max=112,
                                   step=1))

**Entrenamineto**

In [0]:
data_path = 'raw/'
train_data_path = os.path.join(data_path, 'train')
images_train_path = os.listdir(train_data_path)

In [0]:
x_train = []
y_train = []

image_rows = int(512/2)
image_cols = int(512/2) 

resize_value = 512

for image_3d_path in tqdm(images_train_path):

  if 'orig' in image_3d_path:
    image_3d = load_image(train_data_path,image_3d_path)
    for k in range(image_3d.shape[2]):
      #flatt_img = image_3d[:,:,k].reshape(resize_value*resize_value)
      x_train.append(image_3d[::2,::2,k])

  
  elif 'liver' in image_3d_path:
    image_3d = load_image(train_data_path,image_3d_path)
    for k in range(image_3d.shape[2]):
      #flatt_img = image_3d[:,:,k].reshape(resize_value*resize_value)
      y_train.append(image_3d[::2,::2,k])

imgs = np.ndarray(
            (len(x_train), image_rows, image_cols), dtype=np.uint8
            )
imgs_mask = np.ndarray(
        (len(y_train), image_rows, image_cols), dtype=np.uint8
        )

for index, img in enumerate(x_train):
    imgs[index, :, :] = img
    
for index, img in enumerate(y_train):
    imgs_mask[index, :, :] = img

x_train = imgs
y_train = imgs_mask

print(x_train.shape)
print(y_train.shape)

**Test**

In [0]:
data_path = 'raw/'
test_data_path = os.path.join(data_path, 'test')
images_test_path = os.listdir(test_data_path)

In [0]:
x_test = []
y_test = []

image_rows = int(512/2)
image_cols = int(512/2) 

for image_3d_path in tqdm(images_test_path):

  if 'orig' in image_3d_path:
    image_3d = load_image(test_data_path,image_3d_path)
    for k in range(image_3d.shape[2]):
      #flatt_img = image_3d[:,:,k].reshape(resize_value*resize_value)
      x_test.append(image_3d[::2,::2,k])

  
  elif 'liver' in image_3d_path:
    image_3d = load_image(test_data_path,image_3d_path)
    for k in range(image_3d.shape[2]):
      #flatt_img = image_3d[:,:,k].reshape(resize_value*resize_value)
      y_test.append(image_3d[::2,::2,k])

imgs = np.ndarray(
            (len(x_test), image_rows, image_cols), dtype=np.uint8
            )
imgs_mask = np.ndarray(
        (len(y_test), image_rows, image_cols), dtype=np.uint8
        )

for index, img in enumerate(x_test):
    imgs[index, :, :] = img
    
for index, img in enumerate(y_test):
    imgs_mask[index, :, :] = img

x_test = imgs
y_test = imgs_mask

print(x_test.shape)
print(y_test.shape)


(935, 256, 256)
(935, 256, 256)


In [0]:
def preprocess(imgs):
    img_rows = int(512/2)
    img_cols = int(512/2) 
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        imgs_p[i] = resize(imgs[i], (img_cols, img_rows), preserve_range=True)

    imgs_p = imgs_p[..., np.newaxis]
    return imgs_p

In [0]:
salida = preprocess(x_test)
salida.shape

(935, 256, 256, 1)

#### Modelos

##### CNN

**Supuesto:** A partir del conjunto de datos **liver** en un primer momento se pretende reducir la informacion de imagenes binarias que indica por cada pixel si se encuentra el higado presente a etiquetas,donde por cada conjunto de datos de pacientes para una imagen de resonancia magnetica asignar una etiqueta de si se encuentra o no el higado presente en toda la imagen.

In [0]:
data_path = 'raw/test/'
example_filename = os.path.join(data_path, 'ircad_e20_liver.nii.gz')
img_orig = nib.load(example_filename)
img_orig.shape

(512, 512, 225)

In [0]:
data_liver = img_orig.get_fdata()
data_liver[:,:,125].mean()

0.0

##### U-Net

In [0]:
smooth = 1.
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [0]:
def get_unet():
    img_rows = int(512/2)
    img_cols = int(512/2) 
    inputs = Input((img_rows, img_cols, 1))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])

    model.compile(optimizer=Adam(lr=1e-3), loss=dice_coef_loss, metrics=[dice_coef])

    return model


In [0]:
x_train = preprocess(x_train)
y_train = preprocess(y_train)

x_train = x_train.astype('float32')
y_train = y_train.astype('float32')


In [0]:
model = get_unet()
model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)
history=model.fit(x_train, y_train, batch_size=10, epochs=20, verbose=1, shuffle=True,
              validation_split=0.2,
              callbacks=[model_checkpoint])

## Referencias:

- [Application of Deep Learning to the segmentation of medical images](https://github.com/soribadiaby/Deep-Learning-liver-segmentation-project)
- [U-Net: Convolutional Networks for Biomedical
Image Segmentation](https://arxiv.org/pdf/1505.04597.pdf)
- [Data](https://www.dropbox.com/s/8h2avwtk8cfzl49/ircad-dataset.zip?dl=0)
- [Nibabel library](https://nipy.org/nibabel/)